In [48]:
import sys
sys.path.append('/home/UR/kontaous/Bureau/Stage/')
from Analyses.traitement import *
import pandas as pd
import numpy as np
from Analyses.traitement import *
tracking = pd.read_csv("../../../Tracking_data/MontPellier.csv")
events = pd.read_csv("../../Data/match_data/match_Montpellier.csv")
tracking, events = prepare_data(tracking, events)
tracking = tracking.sort_values(by="system_time")
tracking.set_index('frame', inplace=True)
tracking.drop(['Gx', 'Gy', 'id', 'num', 'player'], axis=1, inplace=True)
tracking.rename(columns={'player_name': 'player.name', 'team': 'team.name', 'x': 'location.x', 'y': 'location.y', }, inplace=True)
data_tracking = "../../../Tracking_data/new_MontPellier.csv"
sauvegarder_dataframe_en_csv(tracking, data_tracking)

DataFrame sauvegardé en tant que ../../../Tracking_data/new_MontPellier.csv


In [49]:
df_passes = filtrer_et_formater_passes(events)
data_passes = "../../Data/Tracking/montpellier_passes.csv"
sauvegarder_dataframe_en_csv(df_passes, data_passes)

DataFrame sauvegardé en tant que ../../Data/Tracking/montpellier_passes.csv


In [50]:
df_passes = pd.read_csv('../../Data/Tracking/montpellier_passes.csv')
df_passes

,datetime,team.name,period,player.name,pass.recipient.name,pass.length
0,2023-08-13 00:00:00.474,Montpellier,1,Téji Savanier,Jordan Ferri,11.433723
1,2023-08-13 00:00:01.066,Le Havre,2,Nabil Alioui,Oussama Targhalline,11.815245
2,2023-08-13 00:00:02.175,Montpellier,1,Jordan Ferri,Falaye Sacko,22.375880
3,2023-08-13 00:00:02.815,Le Havre,2,Oussama Targhalline,Issa Soumaré,52.500000
4,2023-08-13 00:00:05.783,Montpellier,1,Falaye Sacko,Christopher Jullien,15.508062
...,...,...,...,...,...,...
906,2023-08-13 00:51:58.203,Le Havre,1,Arthur Desmas,Issa Soumaré,57.215470
907,2023-08-13 00:52:01.596,Montpellier,2,Christopher Jullien,Falaye Sacko,16.046183
908,2023-08-13 00:52:01.675,Montpellier,1,Christopher Jullien,Akor Jerome Adams,26.931395
909,2023-08-13 00:52:40.510,Montpellier,1,Jordan Ferri,Wahbi Khazri,9.948870


In [51]:
df_positions = pd.read_csv('../../../Tracking_data/new_MontPellier.csv')
df_positions

,datetime,system_time,period,team.name,location.x,location.y,player.name,role
0,2023-08-13 15:00:00.000,1691938800000,1,ball,52.770,33.050,ball,ball
1,2023-08-13 15:00:00.000,1691938800000,1,Montpellier,52.309,5.694,Wahbi Khazri,Attaquant
2,2023-08-13 15:00:00.000,1691938800000,1,HAC,63.570,12.992,Loic Nego,Défenseur
3,2023-08-13 15:00:00.000,1691938800000,1,Montpellier,43.696,35.898,Jordan Ferri,Milieu de terrain
4,2023-08-13 15:00:00.000,1691938800000,1,HAC,62.165,33.509,Oussama Targhalline,Milieu de terrain
...,...,...,...,...,...,...,...,...
1448368,2023-08-13 16:52:05.900,1691945525900,2,Montpellier,77.381,14.365,Christopher Jullien,Défenseur
1448369,2023-08-13 16:52:05.900,1691945525900,2,HAC,43.649,28.209,Abdoulaye Toure,Milieu de terrain
1448370,2023-08-13 16:52:05.900,1691945525900,2,Montpellier,75.691,40.296,Boubakar Dit Kiki Kouyate,Défenseur
1448371,2023-08-13 16:52:05.900,1691945525900,2,Montpellier,58.011,18.597,Leo Leroy,Milieu de terrain


In [52]:
import pandas as pd

# Création d'un DataFrame pour les positions des joueurs
data = {
    'role': ["Gardien","Defenseur gauche", "Defenseur droit", "Lateral gauche",  "Lateral droit", "Milieu def", "Milieu off", "Milieu off", "Ailier gauche", "Ailier droit", "Attaquant"],
    'player.name': ['Arthur Desmas', 'Gautier Lloris', 'Loic Nego', 'Arouna Sangante', 'Yoann Salmier', 'Oussama Targhalline', 'Rassoul Ndiaye', 'Daler Kuzyaev',  'Nabil Alioui', 'Josue Casimir', 'Issa Soumarelaye',],
    'team.name': ['Le Havre', 'Le Havre', 'Le Havre','Le Havre','Le Havre','Le Havre','Le Havre','Le Havre','Le Havre','Le Havre','Le Havre'],
    'period': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'location.x': [3, 12, 12, 25, 25, 50, 80, 80, 100, 100, 110],
    'location.y': [40, 30, 50, 10, 70, 40, 20, 60, 10, 70, 40]
}

df_positions = pd.DataFrame(data)

In [56]:
import plotly.graph_objs as go

def visualize_pass_network(player_positions_df, passes_df):
    # Convert dataframes to dictionaries
    players_positions = {
        row['player.name']: (row['location.x'], row['location.y'])
        for index, row in player_positions_df.iterrows()
    }

    # Calculate pass information
    pass_info = passes_df.groupby(['player.name']).agg(
        num_passes=('pass.length', 'size'),
        avg_pass_length=('pass.length', 'mean')
    ).reset_index()

    pass_info_received = passes_df.groupby(['pass.recipient.name']).agg(
        num_passes_received=('pass.length', 'size'),
        avg_pass_length_received=('pass.length', 'mean')
    ).reset_index()

    # Create dictionaries for pass info
    pass_info_dict = {
        row['player.name']: {
            'num_passes': row['num_passes'],
            'avg_pass_length': row['avg_pass_length']
        }
        for index, row in pass_info.iterrows()
    }

    pass_info_received_dict = {
        row['pass.recipient.name']: {
            'num_passes_received': row['num_passes_received'],
            'avg_pass_length_received': row['avg_pass_length_received']
        }
        for index, row in pass_info_received.iterrows()
    }

    # Trace football field
    fig = go.Figure()

    # Add shapes to represent football field
    terrain_shapes = [
        dict(type="rect", x0=0, y0=0, x1=120, y1=80, line=dict(color="Black")),
        dict(type="line", x0=60, y0=0, x1=60, y1=80, line=dict(color="Black", dash="dot")),
        dict(type="rect", x0=0, y0=18, x1=18, y1=62, line=dict(color="Black")),
        dict(type="rect", x0=102, y0=18, x1=120, y1=62, line=dict(color="Black")),
        dict(type="rect", x0=0, y0=30, x1=5.5, y1=50, line=dict(color="Black")),
        dict(type="rect", x0=114.5, y0=30, x1=120, y1=50, line=dict(color="Black")),
        dict(type="rect", x0=0, y0=36, x1=2, y1=44, line=dict(color="Black")),
        dict(type="rect", x0=118, y0=36, x1=120, y1=44, line=dict(color="Black")),
        dict(type="circle", xref="x", yref="y", x0=50.4, y0=30.4, x1=69.6, y1=49.6, line_color="Black"),
    ]
    fig.update_layout(shapes=terrain_shapes)

    # Add players on the field with pass info
    for player, pos in players_positions.items():
        passes_from_player = pass_info_dict.get(player, {'num_passes': 0, 'avg_pass_length': 0})
        passes_to_player = pass_info_received_dict.get(player, {'num_passes_received': 0, 'avg_pass_length_received': 0})

        hover_text = (
            f"Joueur: {player}<br>"
            f"Nombre de passes effectuées: {passes_from_player['num_passes']}<br>"
            f"Longueur moyenne des passes effectuées: {passes_from_player['avg_pass_length']:.2f} m<br>"
            f"Nombre de passes reçues: {passes_to_player['num_passes_received']}<br>"
            f"Longueur moyenne des passes reçues: {passes_to_player['avg_pass_length_received']:.2f} m"
        )

        fig.add_trace(go.Scatter(
            x=[pos[0]],
            y=[pos[1]],
            mode='markers+text',
            text=[player],
            textposition="top center",
            marker=dict(size=10, color='blue'),
            hovertext=hover_text,
            hoverinfo="text"
        ))

    # Calculate number and average length of passes between players
    passes_between_players = passes_df.groupby(['player.name', 'pass.recipient.name']).agg(
        num_passes=('pass.length', 'size'),
        avg_pass_length=('pass.length', 'mean')
    ).reset_index()

    # Add passes
    for row in passes_between_players.itertuples():
        if row[1] in players_positions and row[2] in players_positions:
            start = players_positions[row[1]]
            end = players_positions[row[2]]
            hover_text = (
                f"Passes de {row[1]} à {row[2]}<br>"
                f"Nombre de passes: {row.num_passes}<br>"
                f"Longueur moyenne des passes: {row.avg_pass_length:.2f} m"
            )
            fig.add_trace(go.Scatter(
                x=[start[0], end[0]],
                y=[start[1], end[1]],
                mode='lines',
                line=dict(width=2, color='red'),
                hovertext=hover_text,
                hoverinfo="text"
            ))

    fig.update_layout(
        title="Réseau de passes sur un terrain de football",
        xaxis=dict(range=[0, 120], showgrid=False, zeroline=False),
        yaxis=dict(range=[0, 80], showgrid=False, zeroline=False),
        width=800,
        height=600
    )
    fig.show()

visualize_pass_network(df_positions, df_passes)
